## ІНІЦІАЛІЗАЦІЯ 

Запускати один раз на початку сесії.

Після певного часу відсутності активності Гугл Колаб сервер відключається. Тому, після повторно підключення цей блок потрібно знов запустити.

In [1]:
from IPython.display import display
import sys, os, psutil

W_ENVS = ['jupyter-lab', 'jupyter-notebook', 
          'google.colab', 'python'] 

def working_env():
    parent_process = psutil.Process().parent().cmdline()[-1]

    if W_ENVS[0] in parent_process:
        w_env = W_ENVS[0]
    elif W_ENVS[1] in parent_process:
        w_env = W_ENVS[1]
    elif W_ENVS[2] in sys.modules:
        w_env = W_ENVS[2]
    else:
        w_env = W_ENVS[3]
    
    print(f'Working environment: {w_env}')

    LoM = sys.modules # List of all available Modules
    LoEM = ['pygeodesy', 'centroid_util_colab'] # List of Extra Modules
    if 'google.colab' == w_env and not all(m in LoM for m in LoEM):
        print('NOT all extra modules imported')
        os.system('pip install PyGeodesy')
        os.system('curl -L -o centroid_util_colab.py https://raw.githubusercontent.com/protw/airscape/master/stavok/gis/centroid/centroid_util_colab.py')
    else:
        print('All extra modules imported')

    return w_env

w_env = working_env()

import webbrowser
from centroid_util_colab import centroid_main

Working environment: jupyter-notebook
All extra modules imported


In [2]:
#@title Параметри розрахунків

format_pattern = [
    "46.7373, 32.8128", 
    "46,7373 / 32,8128", 
    "46.7373 32.8128"]
html_page = 'centroid_map.html'

coord_format = "46.7373, 32.8128" #@param ["46.7373, 32.8128", "46,7373 / 32,8128", "46.7373 32.8128"]
lat_lon_order = 'LatLon' #@param ['LatLon', 'LonLat']
perctl = "76" #@param [50, 65, 75, 95]
zoom_start = 17 #@param {type:"slider", min:11, max:18, step:1}

coord_format_id = []
for i,_ in enumerate(format_pattern):
    if coord_format == format_pattern[i]:
        coord_format_id = i
        break

perctl = float(perctl)

print(f'Формат координат: {coord_format}')
print(f'Ід формата координат: {coord_format_id}')
print(f'Порядок координат: {lat_lon_order}')
print(f'Персентиль: {perctl}')
print(f'Стартовий зум мапи: {zoom_start}')


Формат координат: 46.7373, 32.8128
Ід формата координат: 0
Порядок координат: LatLon
Персентиль: 76.0
Стартовий зум мапи: 17


## Зразки вхідного тексту

Десяткова крапка і кома між значеннями (**coord_format_id = 0**):

>	(44.6857, 33.56173); (44.68599, 33.56555); ...  

Десяткова кома і дріб (slash) між значеннями (**coord_format_id = 1**):

> Lorem ipsum dolor sit 46,736906/ 32,810302 adipiscin, do 46,737364/  32,812807 eiusmod ...

Десяткова крапка і пробіл (**coord_format_id = 2**):

> АБВГД 48.01388889 38.79638889 07:59 АБВГД 48.01416667 38.90555556 07:53 ...

In [3]:
#@title СКОПІЮЙТЕ ВХІДНИЙ ТЕКСТ СЮДИ:

text = '49.698824, 37.142584  (MGRS): 49.698164, 37.151901' #@param {type:"string"}

text

'49.698824, 37.142584  (MGRS): 49.698164, 37.151901'

## Розрахунок

In [4]:
map = centroid_main(text, coord_format_id=coord_format_id, perctl=perctl, \
                    zoom_start=zoom_start, html_page=html_page)

Кількість точок: 2
Центроїд:
49.698494, 37.147243
№, коорд, відстань (м):
1, 49.698824, 37.142584, 337.1
2, 49.698164, 37.151901, 337.1
Середня відстань (м) - 337.1
76.0% точок розміщені у колі діаметром 674.1 м


## Візуалізація

In [12]:
if w_env in ['jupyter-lab', 'jupyter-notebook', 'google.colab']:
    display(map)
else:
    webbrowser.open(html_page)